In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment/dataset/sample_submission.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_885.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_698.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1176.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1215.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_66.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_386.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1026.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_330.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_858.wav
/kaggle/input/shl-intern-hiring-ass

In [2]:
pip install torch transformers datasets pandas soundfile librosa openai-whisper --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datasets import Dataset, Audio

# Load train.csv and test.csv
train_df = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv")
test_df = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv")

# Add full audio paths
train_df["audio_path"] = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/" + train_df["filename"]
test_df["audio_path"] = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/" + test_df["filename"]

# Convert labels to class indices (1.0 → 0, 1.5 → 1, ..., 5.0 → 8)
train_df["label"] = (train_df["label"] * 2 - 2).astype(int)

# Create HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df).cast_column("audio_path", Audio())
test_dataset = Dataset.from_pandas(test_df).cast_column("audio_path", Audio())

In [4]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("Current Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


CUDA Available: True
Current Device: Tesla T4


In [6]:
from datasets import Dataset, Audio
import whisper

# Load datasets
train_dataset = Dataset.from_pandas(train_df).cast_column("audio_path", Audio())
test_dataset = Dataset.from_pandas(test_df).cast_column("audio_path", Audio())

# Transcribe (use small model for GPU efficiency)
model = whisper.load_model("medium")

def transcribe(batch):
    transcripts = []
    for audio in batch["audio_path"]:
        path = audio["path"] if isinstance(audio, dict) else audio
        print("Processing:", path)  # Debug        
        result = model.transcribe(audio["path"])
        transcripts.append(result["text"])
    batch["text"] = transcripts
    return batch

# Apply ASR (batched for speed)
train_dataset = train_dataset.map(transcribe, batched=True, batch_size=16)
test_dataset = test_dataset.map(transcribe, batched=True, batch_size=16)

100%|█████████████████████████████████████| 1.42G/1.42G [00:33<00:00, 46.0MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_1261.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_942.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_1110.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_1024.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_538.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_350.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_64.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_252.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_1304.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_1230.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_133.wav
Proces

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_706.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_800.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_68.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1267.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_683.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1242.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_908.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_888.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_137.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_770.wav
Processing: /kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_735.wav
Processing: /kaggle/

In [7]:
train_df_with_text = train_dataset.to_pandas()
test_df_with_text = test_dataset.to_pandas()

# Save to CSV
train_df_with_text.to_csv("train_transcribed.csv", index=False)
test_df_with_text.to_csv("test_transcribed.csv", index=False)

In [35]:
train_df_with_text["label"] = train_df["label"].values
train_df_with_text.to_csv("train_transcribed.csv", index=False)

In [34]:
train_df_with_text.head(5)

,filename,label,audio_path,text
0,audio_1261.wav,1.0,{'bytes': b'RIFFzU\x1d\x00WAVEfmt \x10\x00\x00...,My favorite hobby is cultivation of plants su...
1,audio_942.wav,1.5,{'bytes': b'RIFFzU\x1d\x00WAVEfmt \x10\x00\x00...,The playground looks like very clear and neat...
2,audio_1110.wav,1.5,{'bytes': b'RIFFzU\x1d\x00WAVEfmt \x10\x00\x00...,మాలికండికండికండికండికండికండికండికండికండికండిక...
3,audio_1024.wav,1.5,{'bytes': b'RIFFzU\x1d\x00WAVEfmt \x10\x00\x00...,My favorite place is in Andhra Pradesh. It is...
4,audio_538.wav,2.0,{'bytes': b'RIFFzU\x1d\x00WAVEfmt \x10\x00\x00...,my favorite places my favorite places my expe...


In [18]:
import torch
import numpy as np
from transformers import BertTokenizer, TrainingArguments, Trainer
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, accuracy_score

# =============================================
# 1. Prepare Data from Existing Transcripts
# =============================================
# Verify we have the transcripts
print("Sample training texts:", train_dataset["text"][:2])
print("Sample labels:", train_dataset["label"][:2])

# =============================================
# 2. Tokenization
# =============================================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# =============================================
# 3. Custom BERT Model with Frozen Weights
# =============================================
from transformers import BertPreTrainedModel, BertModel
import torch.nn as nn

class FrozenBertForGrammarScoring(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        
        # Freeze all BERT layers
        for param in self.bert.parameters():
            param.requires_grad = False
            
        # Unfreeze the last 2 layers and pooler
        for layer in self.bert.encoder.layer[-2:]:
            for param in layer.parameters():
                param.requires_grad = True
        for param in self.bert.pooler.parameters():
            param.requires_grad = True
            
        # Classification head (9 classes: 1.0-5.0 in 0.5 steps)
        self.classifier = nn.Sequential(
            nn.Linear(config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 9)
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.classifier(pooled_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        return {"loss": loss, "logits": logits}

model = FrozenBertForGrammarScoring.from_pretrained("bert-base-uncased").to("cuda")

# Verify freezing
print("BERT frozen except last 2 layers and pooler:")
for name, param in model.named_parameters():
    print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")

# =============================================
# 4. Training Setup with Pearson Optimization
# =============================================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    
    # Convert class indices back to original scores (0→1.0, 1→1.5, etc.)
    pred_scores = 1.0 + (predictions * 0.5)
    true_scores = 1.0 + (labels * 0.5)
    
    return {
        "accuracy": accuracy_score(labels, predictions),
        "pearson": pearsonr(pred_scores, true_scores)[0],
        "mse": mean_squared_error(true_scores, pred_scores)
    }

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="pearson",
    greater_is_better=True,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,  # Use validation split if available
    compute_metrics=compute_metrics,
)

# =============================================
# 5. Train the Model
# =============================================
trainer.train()

# =============================================
# 6. Generate Predictions
# =============================================
test_predictions = trainer.predict(test_dataset)
predicted_classes = np.argmax(test_predictions.predictions, axis=1)
predicted_scores = 1.0 + (predicted_classes * 0.5)  # Convert to 1.0-5.0 scale

# Create submission
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": predicted_scores
})
# submission.to_csv("submission.csv", index=False)
submission_path = "/kaggle/working/submission.csv"
submission.to_csv(submission_path, index=False)
# print("Training complete! Submission saved to submission.csv")
if os.path.exists(submission_path):
    print(f"Submission successfully saved to: {submission_path}")
    print("File preview:")
    print(pd.read_csv(submission_path).head())
else:
    print("Error: File not saved!")

Sample training texts: [' My favorite hobby is cultivation of plants such as gardening offers a rewarding experience with gardening. I can gain immense of plants to plant and the flowers and herbs to vegetables.', ' The playground looks like very clear and neat as there are lot of colorful things like a basketball court for playing or we can do gym swing on swings group of friends together play very well and they love each other']
Sample labels: tensor([0, 1])


Map:   0%|          | 0/444 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Some weights of FrozenBertForGrammarScoring were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


BERT frozen except last 2 layers and pooler:
bert.embeddings.word_embeddings.weight: Frozen
bert.embeddings.position_embeddings.weight: Frozen
bert.embeddings.token_type_embeddings.weight: Frozen
bert.embeddings.LayerNorm.weight: Frozen
bert.embeddings.LayerNorm.bias: Frozen
bert.encoder.layer.0.attention.self.query.weight: Frozen
bert.encoder.layer.0.attention.self.query.bias: Frozen
bert.encoder.layer.0.attention.self.key.weight: Frozen
bert.encoder.layer.0.attention.self.key.bias: Frozen
bert.encoder.layer.0.attention.self.value.weight: Frozen
bert.encoder.layer.0.attention.self.value.bias: Frozen
bert.encoder.layer.0.attention.output.dense.weight: Frozen
bert.encoder.layer.0.attention.output.dense.bias: Frozen
bert.encoder.layer.0.attention.output.LayerNorm.weight: Frozen
bert.encoder.layer.0.attention.output.LayerNorm.bias: Frozen
bert.encoder.layer.0.intermediate.dense.weight: Frozen
bert.encoder.layer.0.intermediate.dense.bias: Frozen
bert.encoder.layer.0.output.dense.weight: Fr

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Pearson,Mse
1,No log,2.096143,0.247748,nan,3.150901
2,No log,2.013901,0.247748,nan,3.150901
3,No log,1.938979,0.310811,0.440079,2.225225
4,No log,1.865320,0.351351,0.596855,1.556306
5,No log,1.794715,0.376126,0.662243,1.268018
6,No log,1.733720,0.385135,0.697103,1.099099
7,No log,1.662381,0.414414,0.765483,0.828829
8,No log,1.604514,0.423423,0.785393,0.740991
9,No log,1.567567,0.427928,0.798590,0.691441
10,No log,1.489531,0.463964,0.833994,0.581081


<ipython-input-18-189c1a281713>:90: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  "pearson": pearsonr(pred_scores, true_scores)[0],
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3161: RuntimeWarning: invalid value encountered in greater
  if operator(metric_value, self.state.best_metric):
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
<ipython-input-18-189c1a281713>:90: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  "pearson": pearsonr(pred_scores, true_scores)[0],
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3161: RuntimeWarning: invalid value encountered in greater
  if operator(metric_value, self.state.best_metric):
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_

SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [36]:
import os

# Delete a file
os.remove("/kaggle/working/train_transcribed.csv")  # replace with your filename


In [58]:
import shutil

try:
    # shutil.rmtree('/kaggle/working/results')
    shutil.rmtree('/kaggle/working/enhanced_results')
    
    print("Successfully removed /kaggle/working/results")
except FileNotFoundError:
    print("Folder doesn't exist")
except Exception as e:
    print(f"Error removing folder: {e}")

Successfully removed /kaggle/working/results


In [21]:
import torch
import numpy as np
from transformers import BertTokenizer, TrainingArguments, Trainer
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import os

# =============================================
# 1. Use Your Existing Class-Based Labels
# =============================================
# Your labels are already converted to classes 0-8 (1.0-5.0 in 0.5 steps)
print("Label samples:", train_dataset["label"][:5])

# =============================================
# 2. Tokenization (Keep Your Existing Code)
# =============================================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# =============================================
# 3. Modified Model for Continuous Output
# =============================================
from transformers import BertPreTrainedModel, BertModel
import torch.nn as nn

class BertForContinuousScoring(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        
        # Freeze all BERT layers except last 2
        for param in self.bert.parameters():
            param.requires_grad = False
        for layer in self.bert.encoder.layer[-2:]:
            for param in layer.parameters():
                param.requires_grad = True
                
        # Modified head - outputs continuous values
        self.scoring_head = nn.Sequential(
            nn.Linear(config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1),
            nn.Sigmoid()  # Constrains output to 0-1
        )
        self.scale = 5.0  # Scale to 0-5 range

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        raw_score = self.scoring_head(pooled_output)
        score = raw_score * self.scale  # Convert to 0-5 range
        
        loss = None
        if labels is not None:
            # Convert class labels back to original 1.0-5.0 scale for regression
            true_scores = 1.0 + (labels.float() * 0.5)
            loss_fct = nn.MSELoss()
            loss = loss_fct(score.squeeze(), true_scores)
        return {"loss": loss, "score": score}

model = BertForContinuousScoring.from_pretrained("bert-base-uncased").to("cuda")

# =============================================
# 4. Training Setup with Pearson Optimization
# =============================================
def compute_metrics(eval_pred):
    scores, labels = eval_pred
    scores = scores.squeeze()
    # Convert class indices back to original 1.0-5.0 scale
    true_scores = 1.0 + (labels * 0.5)
    return {
        "pearson": pearsonr(scores, true_scores)[0],
        "mse": mean_squared_error(true_scores, scores)
    }

training_args = TrainingArguments(
    output_dir="/tmp",  # Temporary, gets discarded
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=64,
    evaluation_strategy="no",  # No evaluation during training
    save_strategy="no",        # Prevent saving checkpoints
    logging_strategy="no",     # Skip logging if you want it cleaner
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,  # Use validation split if available
    compute_metrics=compute_metrics,
)

# =============================================
# 5. Train the Model
# =============================================
trainer.train()

# =============================================
# 6. Generate Continuous Predictions (0-5)
# =============================================
test_predictions = trainer.predict(test_dataset)
predicted_scores = test_predictions.predictions.squeeze()

# Clip to ensure 0-5 range and round to 2 decimals
predicted_scores = np.clip(predicted_scores, 0.0, 5.0)
predicted_scores = np.round(predicted_scores, 2)

# Create submission
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": predicted_scores
})
submission_path = "/kaggle/working/submission.csv"
submission.to_csv(submission_path, index=False)

print(f"Continuous predictions saved to {submission_path}")
print("Sample predictions:")
print(submission.head())

Label samples: tensor([0, 1, 1, 1, 2])


Map:   0%|          | 0/444 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Some weights of BertForContinuousScoring were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['scoring_head.0.bias', 'scoring_head.0.weight', 'scoring_head.3.bias', 'scoring_head.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss


TypeError: Unsupported types (<class 'NoneType'>) passed to `_pad_across_processes`. Only nested list/tuple/dicts of objects that are valid for `is_torch_tensor` should be passed.

In [22]:
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TrainingArguments, Trainer
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import os

# =============================================
# 1. Convert Class Labels Back to 0-5 Scores
# =============================================
train_df["label"] = 1.0 + (train_df["label"] * 0.5)  # Convert 0→1.0, 1→1.5, ..., 8→5.0
train_dataset = Dataset.from_pandas(train_df)

# =============================================
# 2. Tokenization
# =============================================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# =============================================
# 3. Regression Model
# =============================================
from transformers import BertPreTrainedModel, BertModel
import torch.nn as nn

class BertForRegression(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.regressor = nn.Sequential(
            nn.Linear(config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        self.scale = 5.0

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        score = self.regressor(pooled_output) * self.scale
        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(score.squeeze(), labels)
        return {"loss": loss, "score": score}

model = BertForRegression.from_pretrained("bert-base-uncased").to("cuda")

# =============================================
# 4. Training Setup (No Checkpoint Saving)
# =============================================
training_args = TrainingArguments(
    output_dir="./tmp",  # Temporary directory
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=30,
    evaluation_strategy="epoch",
    save_strategy="no",  # Disable checkpoint saving
    logging_dir="./logs",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=lambda eval_pred: {
        "pearson": pearsonr(eval_pred.predictions.squeeze(), eval_pred.label_ids)[0],
        "mse": mean_squared_error(eval_pred.label_ids, eval_pred.predictions.squeeze())
    },
)

# =============================================
# 5. Train and Save Only Final CSV
# =============================================
# Train without saving checkpoints
trainer.train()

# Generate predictions
test_predictions = trainer.predict(test_dataset)
predicted_scores = np.clip(test_predictions.predictions.squeeze(), 0.0, 5.0).round(2)

# Create clean submission
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": predicted_scores
})

# Save final CSV
submission_path = "/kaggle/working/submission.csv"
submission.to_csv(submission_path, index=False)


print(f"Final submission saved to {submission_path}")
print(submission.head())

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

KeyError: 'text'

In [29]:
print(train_dataset)


Dataset({
    features: ['filename', 'label', 'audio_path'],
    num_rows: 444
})


In [26]:
import torch
import numpy as np
from transformers import BertTokenizer, TrainingArguments, Trainer
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

# =============================================
# 1. Tokenization
# =============================================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Make sure label is float (for regression)
train_dataset = train_dataset.cast_column("label", "float")

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# =============================================
# 2. Regression Model
# =============================================
from transformers import BertPreTrainedModel, BertModel
import torch.nn as nn

class BertRegressionModel(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)

        for param in self.bert.parameters():
            param.requires_grad = False
        for layer in self.bert.encoder.layer[-2:]:
            for param in layer.parameters():
                param.requires_grad = True
        for param in self.bert.pooler.parameters():
            param.requires_grad = True

        self.regressor = nn.Sequential(
            nn.Linear(config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1)  # Regression output
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        scores = self.regressor(pooled_output).squeeze(-1)

        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(scores, labels)
        return {"loss": loss, "logits": scores}

model = BertRegressionModel.from_pretrained("bert-base-uncased").to("cuda")

# =============================================
# 3. Metrics
# =============================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.clip(predictions.flatten(), 1.0, 5.0)
    labels = labels.flatten()
    return {
        "pearson": pearsonr(predictions, labels)[0],
        "mse": mean_squared_error(labels, predictions)
    }

# =============================================
# 4. TrainingArguments (temp dir, no save)
# =============================================
training_args = TrainingArguments(
    output_dir="/tmp",  # dummy path; won't use it
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=64,
    evaluation_strategy="epoch",
    save_strategy="no",  # <- disables saving
    logging_strategy="epoch",
    load_best_model_at_end=False,
    report_to="none",
    fp16=True,
)

# =============================================
# 5. Trainer
# =============================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    compute_metrics=compute_metrics,
)

# =============================================
# 6. Train
# =============================================
trainer.train()

# =============================================
# 7. Predict
# =============================================
test_predictions = trainer.predict(test_dataset)
predicted_scores = np.clip(test_predictions.predictions.flatten(), 1.0, 5.0)

# Create submission
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": predicted_scores
})
print("Sample predictions:")
print(submission.head())
import os
import pandas as pd

# Define path to save CSV
submission_path = "/kaggle/working/submission.csv"

# Create submission DataFrame
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": predicted_scores
})

# Save to CSV
submission.to_csv(submission_path, index=False)

# Confirm it's saved
if os.path.exists(submission_path):
    print(f"✅ Submission saved to: {submission_path}")
    print(submission.head())
else:
    print("❌ Error: File not saved!")


Map:   0%|          | 0/444 [00:00<?, ? examples/s]

KeyError: 'text'

### Approach -1 (submitted) (documentation needed)(so submit again)

In [50]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, TrainingArguments, Trainer, BertPreTrainedModel, BertModel
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import torch.nn as nn



# Convert to HuggingFace datasets
hf_train_dataset = Dataset.from_pandas(train_df_with_text)
hf_test_dataset = Dataset.from_pandas(test_df_with_text)

# =============================================
# 2. Tokenization
# =============================================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

hf_train_dataset = hf_train_dataset.map(tokenize, batched=True)
hf_test_dataset = hf_test_dataset.map(tokenize, batched=True)

hf_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
hf_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# =============================================
# 3. BERT Model for Regression (Frozen Base)
# =============================================
class BertRegressionModel(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)

        # Freeze all layers
        for param in self.bert.parameters():
            param.requires_grad = False

        # Unfreeze last 2 encoder layers + pooler
        for layer in self.bert.encoder.layer[-2:]:
            for param in layer.parameters():
                param.requires_grad = True
        for param in self.bert.pooler.parameters():
            param.requires_grad = True

        self.regressor = nn.Sequential(
            nn.Linear(config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1)
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.regressor(pooled_output).squeeze(-1)

        loss = None
        if labels is not None:
            loss_fn = nn.MSELoss()
            loss = loss_fn(logits, labels.float())

        return {"loss": loss, "logits": logits}

model = BertRegressionModel.from_pretrained("bert-base-uncased").to("cuda")

# =============================================
# 4. Metrics (Regression + Accuracy)
# =============================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.flatten()
    labels = labels.flatten()
    predictions = np.clip(predictions, 0, 5)

    # Accuracy after rounding to nearest 0.5
    pred_rounded = np.round(predictions * 2) / 2
    label_rounded = np.round(labels * 2) / 2
    accuracy = np.mean(pred_rounded == label_rounded)

    return {
        "mse": mean_squared_error(labels, predictions),
        "pearson": pearsonr(predictions, labels)[0],
        "accuracy": accuracy
    }

# =============================================
# 5. Training Arguments
# =============================================
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=64,
    evaluation_strategy="epoch",
    save_strategy="no",
    # load_best_model_at_end=True,
    # metric_for_best_model="pearson",
    # greater_is_better=True,
    fp16=True,
    report_to="none"
)

# =============================================
# 6. Trainer Setup
# =============================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_train_dataset,  # Use validation set if available
    compute_metrics=compute_metrics,
)

# =============================================
# 7. Train the Model
# =============================================
trainer.train()

# # =============================================
# # 8. Inference on Test Set
# # =============================================
# predictions = trainer.predict(hf_test_dataset)
# predicted_scores = np.clip(predictions.predictions.flatten(), 0, 5)

# # =============================================
# # 9. Submission File
# # =============================================
# submission = pd.DataFrame({
#     "filename": test_df["filename"],
#     "label": predicted_scores
# })

# submission_path = "/kaggle/working/submission.csv"
# submission.to_csv(submission_path, index=False)

# if os.path.exists(submission_path):
#     print(f"✅ Submission saved to: {submission_path}")
#     print(submission.head())
# else:
#     print("❌ Error: Submission file not saved!")

# =============================================
# 8. Inference on Test Set (Corrected)
# =============================================

# 1. First, make sure the test dataset has the right format (only input features)
hf_test_dataset.set_format(
    type="torch", 
    columns=["input_ids", "attention_mask"]  # No labels needed
)

# 2. Create a custom prediction function
def predict_without_labels(model, dataset):
    model.eval()
    predictions = []
    
    # Create dataloader
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=training_args.per_device_eval_batch_size
    )
    
    with torch.no_grad():
        for batch in dataloader:
            # Move batch to GPU
            inputs = {
                'input_ids': batch['input_ids'].to('cuda'),
                'attention_mask': batch['attention_mask'].to('cuda')
            }
            
            # Forward pass
            outputs = model(**inputs)
            predictions.append(outputs['logits'].cpu().numpy())
    
    return np.concatenate(predictions, axis=0)

# 3. Run prediction
predicted_scores = predict_without_labels(model, hf_test_dataset)
predicted_scores = np.clip(predicted_scores.flatten(), 0, 5)
# =============================================
# 9. Submission File
# =============================================
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": predicted_scores
})

submission_path = "/kaggle/working/submission.csv"
submission.to_csv(submission_path, index=False)

if os.path.exists(submission_path):
    print(f"✅ Submission saved to: {submission_path}")
    print(submission.head())
else:
    print("❌ Error: Submission file not saved!")

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Some weights of BertRegressionModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['regressor.0.bias', 'regressor.0.weight', 'regressor.3.bias', 'regressor.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse,Pearson,Accuracy
1,No log,9.901956,9.901956,0.065033,0.000000
2,No log,7.058571,7.058571,0.099318,0.002252
3,No log,4.998337,4.998336,0.080044,0.006757
4,No log,3.436476,3.436476,0.077346,0.157658
5,No log,2.316699,2.316698,0.076595,0.090090
6,No log,1.603327,1.603326,0.090771,0.195946
7,No log,1.264351,1.264351,0.132564,0.051802
8,No log,1.199672,1.199672,0.566352,0.137387
9,No log,0.922330,0.922330,0.652656,0.135135
10,No log,0.680618,0.680618,0.688186,0.195946


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

✅ Submission saved to: /kaggle/working/submission.csv
         filename     label
0   audio_706.wav  1.922620
1   audio_800.wav  2.361012
2    audio_68.wav  4.592855
3  audio_1267.wav  2.886061
4   audio_683.wav  2.682829


In [48]:

print("Test dataset features:", hf_test_dataset.features)

Test dataset features: {'filename': Value(dtype='string', id=None), 'audio_path': {'bytes': Value(dtype='binary', id=None), 'path': Value(dtype='null', id=None)}, 'text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


### Approach -2

In [51]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, TrainingArguments, Trainer, BertPreTrainedModel, BertModel
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import torch.nn as nn
from transformers.modeling_outputs import SequenceClassifierOutput


# Convert to HuggingFace datasets
hf_train_dataset = Dataset.from_pandas(train_df_with_text)
hf_test_dataset = Dataset.from_pandas(test_df_with_text)

# =============================================
# 2. Tokenization
# =============================================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

hf_train_dataset = hf_train_dataset.map(tokenize, batched=True)
hf_test_dataset = hf_test_dataset.map(tokenize, batched=True)

hf_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
hf_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# =============================================
# 3. BERT Model for Regression (Frozen Base)
# =============================================
class BertRegressionModel(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)

        # Freeze all BERT layers
        for param in self.bert.parameters():
            param.requires_grad = False

        # Unfreeze last 4 layers and pooler
        for layer in self.bert.encoder.layer[-4:]:
            for param in layer.parameters():
                param.requires_grad = True
        for param in self.bert.pooler.parameters():
            param.requires_grad = True

        self.regressor = nn.Sequential(
            nn.Linear(config.hidden_size, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 1)
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        # Mean pooling instead of CLS token
        last_hidden_state = outputs.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        mean_pooled = sum_embeddings / sum_mask

        logits = self.regressor(mean_pooled).squeeze(-1)

        loss = None
        if labels is not None:
            loss_fn = nn.MSELoss()
            loss = loss_fn(logits, labels.float())

        return SequenceClassifierOutput(loss=loss, logits=logits)
model = BertRegressionModel.from_pretrained("bert-base-uncased").to("cuda")

# =============================================
# 4. Metrics (Regression + Accuracy)
# =============================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.flatten()
    labels = labels.flatten()
    predictions = np.clip(predictions, 0, 5)

    # Accuracy after rounding to nearest 0.5
    pred_rounded = np.round(predictions * 2) / 2
    label_rounded = np.round(labels * 2) / 2
    accuracy = np.mean(pred_rounded == label_rounded)

    return {
        "mse": mean_squared_error(labels, predictions),
        "pearson": pearsonr(predictions, labels)[0],
        "accuracy": accuracy
    }

# =============================================
# 5. Training Arguments
# =============================================
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=64,
    evaluation_strategy="epoch",
    save_strategy="no",
    # load_best_model_at_end=True,
    # metric_for_best_model="pearson",
    # greater_is_better=True,
    fp16=True,
    report_to="none"
)

# =============================================
# 6. Trainer Setup
# =============================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_train_dataset,  # Use validation set if available
    compute_metrics=compute_metrics,
)

# =============================================
# 7. Train the Model
# =============================================
trainer.train()

# # =============================================
# # 8. Inference on Test Set
# # =============================================
# predictions = trainer.predict(hf_test_dataset)
# predicted_scores = np.clip(predictions.predictions.flatten(), 0, 5)

# # =============================================
# # 9. Submission File
# # =============================================
# submission = pd.DataFrame({
#     "filename": test_df["filename"],
#     "label": predicted_scores
# })

# submission_path = "/kaggle/working/submission.csv"
# submission.to_csv(submission_path, index=False)

# if os.path.exists(submission_path):
#     print(f"✅ Submission saved to: {submission_path}")
#     print(submission.head())
# else:
#     print("❌ Error: Submission file not saved!")

# =============================================
# 8. Inference on Test Set (Corrected)
# =============================================

# 1. First, make sure the test dataset has the right format (only input features)
hf_test_dataset.set_format(
    type="torch", 
    columns=["input_ids", "attention_mask"]  # No labels needed
)

# 2. Create a custom prediction function
def predict_without_labels(model, dataset):
    model.eval()
    predictions = []
    
    # Create dataloader
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=training_args.per_device_eval_batch_size
    )
    
    with torch.no_grad():
        for batch in dataloader:
            # Move batch to GPU
            inputs = {
                'input_ids': batch['input_ids'].to('cuda'),
                'attention_mask': batch['attention_mask'].to('cuda')
            }
            
            # Forward pass
            outputs = model(**inputs)
            predictions.append(outputs['logits'].cpu().numpy())
    
    return np.concatenate(predictions, axis=0)

# 3. Run prediction
predicted_scores = predict_without_labels(model, hf_test_dataset)
predicted_scores = np.clip(predicted_scores.flatten(), 0, 5)
# =============================================
# 9. Submission File
# =============================================
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": predicted_scores
})

submission_path = "/kaggle/working/submission.csv"
submission.to_csv(submission_path, index=False)

if os.path.exists(submission_path):
    print(f"✅ Submission saved to: {submission_path}")
    print(submission.head())
else:
    print("❌ Error: Submission file not saved!")

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Some weights of BertRegressionModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['regressor.0.bias', 'regressor.0.weight', 'regressor.1.bias', 'regressor.1.weight', 'regressor.4.bias', 'regressor.4.weight', 'regressor.5.bias', 'regressor.5.weight', 'regressor.8.bias', 'regressor.8.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse,Pearson,Accuracy
1,No log,3.714662,3.714661,0.139329,0.150901
2,No log,2.062696,2.062696,0.053326,0.092342
3,No log,1.586926,1.586925,0.139561,0.195946
4,No log,1.424692,1.424692,0.151428,0.195946
5,No log,1.345661,1.345661,0.177533,0.051802
6,No log,1.305839,1.305839,0.206875,0.051802
7,No log,1.278975,1.278975,0.210682,0.051802
8,No log,1.258077,1.258077,0.271223,0.051802
9,No log,1.228281,1.228281,0.361323,0.051802
10,No log,1.124773,1.124773,0.557352,0.087838


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

✅ Submission saved to: /kaggle/working/submission.csv
         filename     label
0   audio_706.wav  1.899576
1   audio_800.wav  1.972397
2    audio_68.wav  3.915667
3  audio_1267.wav  3.184306
4   audio_683.wav  2.690701


### Approach -3

In [56]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, TrainingArguments, Trainer, BertPreTrainedModel, BertModel
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import torch.nn as nn



# Convert to HuggingFace datasets
hf_train_dataset = Dataset.from_pandas(train_df_with_text)
hf_test_dataset = Dataset.from_pandas(test_df_with_text)

# =============================================
# 2. Tokenization
# =============================================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

hf_train_dataset = hf_train_dataset.map(tokenize, batched=True)
hf_test_dataset = hf_test_dataset.map(tokenize, batched=True)

hf_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
hf_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# =============================================
# 3. BERT Model for Regression (Frozen Base)
# =============================================
class BertRegressionModel(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        
        # Freeze all layers initially
        for param in self.bert.parameters():
            param.requires_grad = False
            
        # Unfreeze last 3 encoder layers + pooler
        for layer in self.bert.encoder.layer[-3:]:
            for param in layer.parameters():
                param.requires_grad = True
        for param in self.bert.pooler.parameters():
            param.requires_grad = True
            
        # Enhanced regression head with layer normalization and more capacity
        self.regressor = nn.Sequential(
            nn.Linear(config.hidden_size, 512),  # Note: Using single hidden_size now
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.2),
            
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.GELU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(0.1),
            
            nn.Linear(128, 1)
        )
        
        self.init_weights()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        
        # Using just the last hidden state's [CLS] token representation
        # (equivalent to pooler_output but gives you more control)
        last_hidden_state = outputs.last_hidden_state
        cls_token = last_hidden_state[:, 0, :]  # Take [CLS] token
        
        logits = self.regressor(cls_token).squeeze(-1)
        
        loss = None
        if labels is not None:
            loss_fn = nn.SmoothL1Loss()
            loss = loss_fn(logits, labels.float())
            
            # L2 regularization
            l2_lambda = 0.01
            l2_norm = sum(p.pow(2.0).sum() for p in self.regressor.parameters())
            loss = loss + l2_lambda * l2_norm

        return {"loss": loss, "logits": logits}

model = BertRegressionModel.from_pretrained("bert-base-uncased").to("cuda")

# =============================================
# 4. Metrics (Regression + Accuracy)
# =============================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.flatten()
    labels = labels.flatten()
    predictions = np.clip(predictions, 0, 5)

    # Accuracy after rounding to nearest 0.5
    pred_rounded = np.round(predictions * 2) / 2
    label_rounded = np.round(labels * 2) / 2
    accuracy = np.mean(pred_rounded == label_rounded)

    return {
        "mse": mean_squared_error(labels, predictions),
        "pearson": pearsonr(predictions, labels)[0],
        "accuracy": accuracy
    }

# =============================================
# 5. Training Arguments
# =============================================
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,  # Larger batch for evaluation
    num_train_epochs=40,            # Reduced epochs (can use early stopping)
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="pearson",
    greater_is_better=True,
    fp16=True,
    report_to="none",
    learning_rate=3e-5,             # Adjusted learning rate
    weight_decay=0.01,              # Explicit weight decay
    warmup_ratio=0.1,               # Learning rate warmup
    gradient_accumulation_steps=2,  # For more stable training
    save_total_limit=2,             # Keep only best models
)

# =============================================
# 6. Trainer Setup
# =============================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_train_dataset,  # Use validation set if available
    compute_metrics=compute_metrics,
)

# =============================================
# 7. Train the Model
# =============================================
trainer.train()

# # =============================================
# # 8. Inference on Test Set
# # =============================================
# predictions = trainer.predict(hf_test_dataset)
# predicted_scores = np.clip(predictions.predictions.flatten(), 0, 5)

# # =============================================
# # 9. Submission File
# # =============================================
# submission = pd.DataFrame({
#     "filename": test_df["filename"],
#     "label": predicted_scores
# })

# submission_path = "/kaggle/working/submission.csv"
# submission.to_csv(submission_path, index=False)

# if os.path.exists(submission_path):
#     print(f"✅ Submission saved to: {submission_path}")
#     print(submission.head())
# else:
#     print("❌ Error: Submission file not saved!")

# =============================================
# 8. Inference on Test Set (Corrected)
# =============================================

# 1. First, make sure the test dataset has the right format (only input features)
hf_test_dataset.set_format(
    type="torch", 
    columns=["input_ids", "attention_mask"]  # No labels needed
)

# 2. Create a custom prediction function
def predict_without_labels(model, dataset):
    model.eval()
    predictions = []
    
    # Create dataloader
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=training_args.per_device_eval_batch_size
    )
    
    with torch.no_grad():
        for batch in dataloader:
            # Move batch to GPU
            inputs = {
                'input_ids': batch['input_ids'].to('cuda'),
                'attention_mask': batch['attention_mask'].to('cuda')
            }
            
            # Forward pass
            outputs = model(**inputs)
            predictions.append(outputs['logits'].cpu().numpy())
    
    return np.concatenate(predictions, axis=0)

# 3. Run prediction
predicted_scores = predict_without_labels(model, hf_test_dataset)
predicted_scores = np.clip(predicted_scores.flatten(), 0, 5)
# =============================================
# 9. Submission File
# =============================================
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": predicted_scores
})

submission_path = "/kaggle/working/submission.csv"
submission.to_csv(submission_path, index=False)

if os.path.exists(submission_path):
    print(f"✅ Submission saved to: {submission_path}")
    print(submission.head())
else:
    print("❌ Error: Submission file not saved!")

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Some weights of BertRegressionModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['regressor.0.bias', 'regressor.0.weight', 'regressor.1.bias', 'regressor.1.weight', 'regressor.12.bias', 'regressor.12.weight', 'regressor.4.bias', 'regressor.4.weight', 'regressor.5.bias', 'regressor.5.weight', 'regressor.8.bias', 'regressor.8.weight', 'regressor.9.bias', 'regressor.9.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse,Pearson,Accuracy
1,2.733400,8.650088,13.788647,0.086535,0.000000
2,2.607400,8.297031,11.515423,-0.129747,0.000000
3,2.364600,7.951439,9.393885,-0.007187,0.002252
4,2.110200,7.709852,8.058847,0.046874,0.002252
5,1.935200,7.549747,7.230450,0.067061,0.002252
6,1.825500,7.440984,6.686299,0.087715,0.004505
7,1.739700,7.362116,6.302498,0.095855,0.006757
8,1.677900,7.301018,6.011431,0.098177,0.006757
9,1.642900,7.253753,5.790096,0.097604,0.006757
10,1.591000,7.217873,5.623953,0.098937,0.006757


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

✅ Submission saved to: /kaggle/working/submission.csv
         filename     label
0   audio_706.wav  1.485572
1   audio_800.wav  1.528615
2    audio_68.wav  1.532290
3  audio_1267.wav  1.522899
4   audio_683.wav  1.527562


### Approach -4

In [63]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, TrainingArguments, Trainer, BertPreTrainedModel, BertModel
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import torch.nn as nn



# Convert to HuggingFace datasets
hf_train_dataset = Dataset.from_pandas(train_df_with_text)
hf_test_dataset = Dataset.from_pandas(test_df_with_text)

# =============================================
# 2. Tokenization
# =============================================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

hf_train_dataset = hf_train_dataset.map(tokenize, batched=True)
hf_test_dataset = hf_test_dataset.map(tokenize, batched=True)

hf_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
hf_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# =============================================
# 3. BERT Model for Regression (Frozen Base)
# =============================================
class BertRegressionModel(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)

        # Freeze all layers
        for param in self.bert.parameters():
            param.requires_grad = False

        # Unfreeze last 2 encoder layers + pooler
        for layer in self.bert.encoder.layer[-2:]:
            for param in layer.parameters():
                param.requires_grad = True
        for param in self.bert.pooler.parameters():
            param.requires_grad = True

        self.regressor = nn.Sequential(
            nn.Linear(config.hidden_size, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(0.3),
        
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(0.3),
        
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
        
            nn.Linear(64, 1)
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.regressor(pooled_output).squeeze(-1)

        loss = None
        if labels is not None:
            loss_fn = nn.MSELoss()
            loss = loss_fn(logits, labels.float())

        return {"loss": loss, "logits": logits}

model = BertRegressionModel.from_pretrained("bert-base-uncased").to("cuda")

# =============================================
# 4. Metrics (Regression + Accuracy)
# =============================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.flatten()
    labels = labels.flatten()
    predictions = np.clip(predictions, 0, 5)

    # Accuracy after rounding to nearest 0.5
    pred_rounded = np.round(predictions * 2) / 2
    label_rounded = np.round(labels * 2) / 2
    accuracy = np.mean(pred_rounded == label_rounded)

    return {
        "mse": mean_squared_error(labels, predictions),
        "pearson": pearsonr(predictions, labels)[0],
        "accuracy": accuracy
    }

# =============================================
# 5. Training Arguments
# =============================================
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=256,
    evaluation_strategy="epoch",
    save_strategy="no",
    # load_best_model_at_end=True,
    # metric_for_best_model="pearson",
    # greater_is_better=True,
    fp16=True,
    report_to="none"
)

# =============================================
# 6. Trainer Setup
# =============================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_train_dataset,  # Use validation set if available
    compute_metrics=compute_metrics,
)

# =============================================
# 7. Train the Model
# =============================================
trainer.train()

# # =============================================
# # 8. Inference on Test Set
# # =============================================
# predictions = trainer.predict(hf_test_dataset)
# predicted_scores = np.clip(predictions.predictions.flatten(), 0, 5)

# # =============================================
# # 9. Submission File
# # =============================================
# submission = pd.DataFrame({
#     "filename": test_df["filename"],
#     "label": predicted_scores
# })

# submission_path = "/kaggle/working/submission.csv"
# submission.to_csv(submission_path, index=False)

# if os.path.exists(submission_path):
#     print(f"✅ Submission saved to: {submission_path}")
#     print(submission.head())
# else:
#     print("❌ Error: Submission file not saved!")

# =============================================
# 8. Inference on Test Set (Corrected)
# =============================================

# 1. First, make sure the test dataset has the right format (only input features)
hf_test_dataset.set_format(
    type="torch", 
    columns=["input_ids", "attention_mask"]  # No labels needed
)

# 2. Create a custom prediction function
def predict_without_labels(model, dataset):
    model.eval()
    predictions = []
    
    # Create dataloader
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=training_args.per_device_eval_batch_size
    )
    
    with torch.no_grad():
        for batch in dataloader:
            # Move batch to GPU
            inputs = {
                'input_ids': batch['input_ids'].to('cuda'),
                'attention_mask': batch['attention_mask'].to('cuda')
            }
            
            # Forward pass
            outputs = model(**inputs)
            predictions.append(outputs['logits'].cpu().numpy())
    
    return np.concatenate(predictions, axis=0)

# 3. Run prediction
predicted_scores = predict_without_labels(model, hf_test_dataset)
predicted_scores = np.clip(predicted_scores.flatten(), 0, 5)
# =============================================
# 9. Submission File
# =============================================
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": predicted_scores
})

submission_path = "/kaggle/working/submission.csv"
submission.to_csv(submission_path, index=False)

if os.path.exists(submission_path):
    print(f"✅ Submission saved to: {submission_path}")
    print(submission.head())
else:
    print("❌ Error: Submission file not saved!")

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Some weights of BertRegressionModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['regressor.0.bias', 'regressor.0.weight', 'regressor.1.bias', 'regressor.1.weight', 'regressor.11.bias', 'regressor.11.weight', 'regressor.4.bias', 'regressor.4.weight', 'regressor.5.bias', 'regressor.5.weight', 'regressor.8.bias', 'regressor.8.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse,Pearson,Accuracy
1,No log,13.370345,13.370344,-0.013619,0.000000
2,No log,12.577592,12.577592,0.025159,0.000000
3,No log,12.019942,12.019942,0.082255,0.000000
4,No log,11.620266,11.620267,0.097893,0.000000
5,No log,11.255843,11.255842,0.080286,0.000000
6,No log,10.923291,10.923291,0.046035,0.000000
7,No log,10.603070,10.603069,0.072765,0.000000
8,No log,10.285638,10.285638,0.089789,0.000000
9,No log,9.973025,9.973027,0.095646,0.000000
10,No log,9.659416,9.659416,0.067053,0.000000


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

✅ Submission saved to: /kaggle/working/submission.csv
         filename     label
0   audio_706.wav  1.828483
1   audio_800.wav  2.256455
2    audio_68.wav  4.376028
3  audio_1267.wav  3.025353
4   audio_683.wav  2.594480


In [64]:
print("hello")

hello


### Approach-5

In [3]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, TrainingArguments, Trainer, BertPreTrainedModel, BertModel
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import torch.nn as nn



# Convert to HuggingFace datasets
hf_train_dataset = Dataset.from_pandas(train_df_with_text)
hf_test_dataset = Dataset.from_pandas(test_df_with_text)

# =============================================
# 2. Tokenization
# =============================================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

hf_train_dataset = hf_train_dataset.map(tokenize, batched=True)
hf_test_dataset = hf_test_dataset.map(tokenize, batched=True)

hf_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
hf_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# =============================================
# 3. BERT Model for Regression (Frozen Base)
# =============================================
class BertRegressionModel(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)

        # Freeze all layers
        for param in self.bert.parameters():
            param.requires_grad = False

        # Unfreeze last 2 encoder layers + pooler
        for layer in self.bert.encoder.layer[-2:]:
            for param in layer.parameters():
                param.requires_grad = True
        for param in self.bert.pooler.parameters():
            param.requires_grad = True

        self.regressor = nn.Sequential(
            nn.Linear(config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1)
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.regressor(pooled_output).squeeze(-1)

        loss = None
        if labels is not None:
            loss_fn = nn.MSELoss()
            loss = loss_fn(logits, labels.float())

        return {"loss": loss, "logits": logits}

model = BertRegressionModel.from_pretrained("bert-base-uncased").to("cuda")

# =============================================
# 4. Metrics (Regression + Accuracy)
# =============================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.flatten()
    labels = labels.flatten()
    predictions = np.clip(predictions, 0, 5)

    # Accuracy after rounding to nearest 0.5
    pred_rounded = np.round(predictions * 2) / 2
    label_rounded = np.round(labels * 2) / 2
    accuracy = np.mean(pred_rounded == label_rounded)

    return {
        "mse": mean_squared_error(labels, predictions),
        "pearson": pearsonr(predictions, labels)[0],
        "accuracy": accuracy
    }

# =============================================
# 5. Training Arguments
# =============================================
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=128,
    evaluation_strategy="epoch",
    save_strategy="no",
    # load_best_model_at_end=True,
    # metric_for_best_model="pearson",
    # greater_is_better=True,
    fp16=True,
    report_to="none"
)

# =============================================
# 6. Trainer Setup
# =============================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_train_dataset,  # Use validation set if available
    compute_metrics=compute_metrics,
)

# =============================================
# 7. Train the Model
# =============================================
trainer.train()

# # =============================================
# # 8. Inference on Test Set
# # =============================================
# predictions = trainer.predict(hf_test_dataset)
# predicted_scores = np.clip(predictions.predictions.flatten(), 0, 5)

# # =============================================
# # 9. Submission File
# # =============================================
# submission = pd.DataFrame({
#     "filename": test_df["filename"],
#     "label": predicted_scores
# })

# submission_path = "/kaggle/working/submission.csv"
# submission.to_csv(submission_path, index=False)

# if os.path.exists(submission_path):
#     print(f"✅ Submission saved to: {submission_path}")
#     print(submission.head())
# else:
#     print("❌ Error: Submission file not saved!")

# =============================================
# 8. Inference on Test Set (Corrected)
# =============================================

# 1. First, make sure the test dataset has the right format (only input features)
hf_test_dataset.set_format(
    type="torch", 
    columns=["input_ids", "attention_mask"]  # No labels needed
)

# 2. Create a custom prediction function
def predict_without_labels(model, dataset):
    model.eval()
    predictions = []
    
    # Create dataloader
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=training_args.per_device_eval_batch_size
    )
    
    with torch.no_grad():
        for batch in dataloader:
            # Move batch to GPU
            inputs = {
                'input_ids': batch['input_ids'].to('cuda'),
                'attention_mask': batch['attention_mask'].to('cuda')
            }
            
            # Forward pass
            outputs = model(**inputs)
            predictions.append(outputs['logits'].cpu().numpy())
    
    return np.concatenate(predictions, axis=0)

# 3. Run prediction
predicted_scores = predict_without_labels(model, hf_test_dataset)
predicted_scores = np.clip(predicted_scores.flatten(), 0, 5)
# =============================================
# 9. Submission File
# =============================================
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": predicted_scores
})

submission_path = "/kaggle/working/submission.csv"
submission.to_csv(submission_path, index=False)

if os.path.exists(submission_path):
    print(f"✅ Submission saved to: {submission_path}")
    print(submission.head())
else:
    print("❌ Error: Submission file not saved!")

NameError: name 'train_df_with_text' is not defined